## Descrição do problema

Os cartões de crédito são utilizados na indústria financeira para prever a probabilidade de inadimplência e orientar a emissão de cartões de crédito. Esses scores de crédito dependem dos dados pessoais dos candidatos e de informações históricas, quantificando o risco de maneira objetiva.

Este projeto tem como objetivo criar um modelo de Classificador de Floresta Aleatória (Random Forest) para ajudar os bancos a decidir quem deve receber um cartão de crédito.

## Carregando Bibliotecas

In [1]:
# Carregando bibliotecas
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , classification_report, f1_score, precision_score, recall_score

## Descrição das colunas do conjuto de dados "application record"

| Nome do recurso          | Explicação                          | Observações                                                                 |
|--------------------------|-------------------------------------|-----------------------------------------------------------------------------|
| ID                       | Número do cliente                   |                                                                             |
| CODE_GENDER              | Gênero                              |                                                                             |
| FLAG_OWN_CAR             | Tem um carro                        |                                                                             |
| FLAG_OWN_REALTY          | Existe uma propriedade              |                                                                             |
| CNT_CHILDREN             | Número de crianças                  |                                                                             |
| AMT_INCOME_TOTAL         | Renda anual                         |                                                                             |
| NAME_INCOME_TYPE         | Categoria de renda                  |                                                                             |
| NAME_EDUCATION_TYPE      | Nível de educação                   |                                                                             |
| NAME_FAMILY_STATUS       | Estado civil                        |                                                                             |
| NAME_HOUSING_TYPE        | Modo de vida                        |                                                                             |
| DAYS_BIRTH               | Aniversário                         | Contagem regressiva a partir do dia atual (0), -1 significa ontem           |
| DAYS_EMPLOYED            | Data de início do emprego           | Contagem regressiva a partir do dia atual (0). Se positivo, significa que a pessoa está atualmente desempregada. |
| FLAG_MOBIL               | Existe um telefone celular?         |                                                                             |
| FLAG_WORK_PHONE          | Existe um telefone de trabalho?     |                                                                             |
| FLAG_PHONE               | Tem algum telefone?                 |                                                                             |
| FLAG_EMAIL               | Existe um e-mail                    |                                                                             |
| OCCUPATION_TYPE          | Ocupação                            |                                                                             |
| CNT_FAM_MEMBERS          | Tamanho da família                  |                                                                             |


# Carregando conjunto de dados application record

In [2]:
# Carregando o conjuto de dados
df_application = pd.read_csv('../data/application_record.csv')

# Pré-visualizando conjunto de dados
df_application.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
# Conhecendo o conjuto de dados aplplication record 
print('Visualizando quantidade de linhas do conjunto de dados application record:', df_application.shape[0])

print('Visualizando quantidade de colunas do conjunto de dados application record:', df_application.shape[1])

Visualizando quantidade de linhas do conjunto de dados application record: 438557
Visualizando quantidade de colunas do conjunto de dados application record: 18


In [4]:
# Visualizando tipos de dados
df_application.dtypes

ID                       int64
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_MOBIL               int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
dtype: object

### Observações

* O conjunto de dados application record possuí 438557 linhas e 18 colunas
* O conjunto contém um total de 10 colunas numéricas
* O conjuto contém 8 colunas object

## Descrição das colunas do conjunto de dados "credit record"

| Nome do Recurso   | Explicação  | Observações |
|------------------|------------|-------------|
| ID              | Número do cliente | |
| MONTHS_BALANCE | Mês recorde | O mês dos dados extraídos é o ponto de partida, para trás, 0 é o mês atual, -1 é o mês anterior e assim por diante |
| STATUS         | Status | 0: 1-29 dias em atraso <br> 1: 30-59 dias em atraso <br> 2: 60-89 dias em atraso <br> 3: 90-119 dias em atraso <br> 4: 120-149 dias em atraso <br> 5: Dívidas vencidas ou inadimplentes, amortizações por mais de 150 dias <br> C: Quitadas naquele mês <br> X: Nenhum empréstimo no mês |


# Carregando conjunto de dados credit record

In [5]:
# Carregando conjunto de dados
df_credit = pd.read_csv('../data/credit_record.csv')

# Pré-visualizando conjunto de dados
df_credit.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [6]:
print('Visualizando quantidade de linhas do conjunto de dados credit record:', df_credit.shape[0])

print('Visualizando quantidade de colunas do conjunto de dados credit record:', df_credit.shape[1])

Visualizando quantidade de linhas do conjunto de dados credit record: 1048575
Visualizando quantidade de colunas do conjunto de dados credit record: 3


In [7]:
# Visualizando tipos de dados
df_credit.dtypes

ID                 int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object

### Observações

* O conjunto de dados credit record possuí 1048575 linhas e 3 colunas
* O conjunto contém um total de 2 colunas numéricas
* O conjuto contém 1 coluna object que mostra o status dos detalhes da cobrança do cartão de credito